In [1]:
from jellyfish._jellyfish import damerau_levenshtein_distance
import distance
from sklearn import metrics

In [2]:
from keras.models import load_model
from theano.ifelse import ifelse #added this

Using Theano backend.


In [3]:
import pickle
import pandas as pd
import numpy as np
from utils_test import *

In [4]:
test = pd.read_csv('../input/test.csv')
test = transformDf(test)

In [5]:
with open('../input/parameters.pkl', "rb") as f:
    maxlen = pickle.load(f)
    num_features = pickle.load(f)
    chartoindice = pickle.load(f)
    targetchartoindice = pickle.load(f)
    divisor = pickle.load(f)
    divisor2 = pickle.load(f)

In [6]:
with open('../input/preprocessed_data.pkl', "rb") as f:
    X = pickle.load(f)
    y_a = pickle.load(f)
    y_t = pickle.load(f)
    X_test = pickle.load(f)
    y_a_test = pickle.load(f)
    y_t_test = pickle.load(f)

# Utils

In [7]:
model = load_model('output_files/models/model_31-0.95.h5')

/Users/hoangnguyen/miniconda3/envs/pydata2/lib/python2.7/site-packages/theano/tensor/basic.py:5130: UserWarning: flatten outdim parameter is deprecated, use ndim instead.
  "flatten outdim parameter is deprecated, use ndim instead.")


In [8]:
test_groupByCase = test.groupby(['CaseID'])

In [9]:
test_len = findLongestLength(test_groupByCase)
test_len - 5

8

In [10]:
sentences, sentences_t, sentences_t2, sentences_t3, sentences_t4 = getFeature(test_groupByCase)

In [11]:
next_chars, next_chars_t, next_chars_t2, next_chars_t3, next_chars_t4 = getOutput(test_groupByCase)

In [12]:
def getLabel(prediction, targetchartoindice=targetchartoindice):
    indicetotargetchar = dict((indice, char) for char, indice in targetchartoindice.items())
    label_list = []
    for i in range(prediction.shape[0]):
        max_value_index = np.argmax(prediction[i])
        label = indicetotargetchar[max_value_index]
        label_list.append(label)
    return label_list

In [13]:
def inverseTime(predictions, divisor=divisor):
    pred_t = predictions*divisor
    return np.maximum(pred_t, 0)

# Suffix

## Predict

In [81]:
suffix = 2 #Change in the interval [2, 7]

In [69]:
sentences_2 = []
sentences_t_2 = []
sentences_t2_2 = []
sentences_t3_2 = []
sentences_t4_2 = []
next_chars_2 = []
next_chars_t_2 = []

for i in range(len(sentences)):
    if len(sentences[i]) == suffix:
        sentences_2.append(sentences[i])
        sentences_t_2.append(sentences_t[i])
        sentences_t2_2.append(sentences_t2[i])
        sentences_t3_2.append(sentences_t3[i])
        sentences_t4_2.append(sentences_t4[i])
        next_chars_2.append(next_chars[i])
        next_chars_t_2.append(next_chars_t[i])

In [70]:
len(sentences_2), len(next_chars_2)

(89, 89)

In [71]:
X_test = vectorizeInput(sentences_2, sentences_t_2, sentences_t2_2, sentences_t3_2, sentences_t4_2, 
                        maxlen, num_features, chartoindice, 
                        divisor, divisor2, divisor3=86400, divisor4=7)

In [72]:
pred = model.predict(X_test, verbose=0)
pred_a = pred[0]
pred_t = pred[1]

In [73]:
pred[0].shape, pred[1].shape

((89, 10), (89, 1))

## Evaluate

### Activity

In [74]:
#for log-loss
y_a_test = one_hot_encode(next_chars_2, targetchartoindice)

In [75]:
metrics.log_loss(y_a_test, pred_a)

0.37326596068281137

In [76]:
#for accuracy
gt_a_label = next_chars_2
pred_a_label = getLabel(pred_a)

In [77]:
metrics.accuracy_score(gt_a_label, pred_a_label)

0.898876404494382

### Time

In [78]:
y_t_test = normalize(next_chars_t_2, divisor)

In [79]:
gt_t = next_chars_t_2

In [80]:
metrics.mean_squared_error(gt_t, pred_t)

161539365284.58835